In [1]:
import gurobipy as gp
from gurobipy import GRB

# Example definitions for M, N, setup_times, and processing_times
# Replace these with actual data
M = range(3)  # Example: 3 machines
N = range(5)  # Example: 5 jobs
N0 = [0] + list(N)  # Including dummy job 0

# Define U as a sufficiently large number
U = 10000

# Example multidimensional lists for setup times and processing times
# Ensure these lists are correctly defined for your problem

# setup_times[i][j][k] for machine i, job j to job k
setup_times = [
    [[0, 1, 2, 3, 4, 5], [1, 0, 2, 3, 4, 5], [2, 2, 0, 3, 4, 5], [3, 3, 3, 0, 4, 5], [4, 4, 4, 4, 0, 5], [5, 5, 5, 5, 5, 0]],
    [[0, 1, 2, 3, 4, 5], [1, 0, 2, 3, 4, 5], [2, 2, 0, 3, 4, 5], [3, 3, 3, 0, 4, 5], [4, 4, 4, 4, 0, 5], [5, 5, 5, 5, 5, 0]],
    [[0, 1, 2, 3, 4, 5], [1, 0, 2, 3, 4, 5], [2, 2, 0, 3, 4, 5], [3, 3, 3, 0, 4, 5], [4, 4, 4, 4, 0, 5], [5, 5, 5, 5, 5, 0]]
]

# processing_times[i][k] for machine i, job k
processing_times = [
    [1, 2, 3, 4, 5],
    [2, 3, 4, 5, 6],
    [3, 4, 5, 6, 7]
]

# Create model
model = gp.Model('Scheduling')

# Decision variables
y = model.addVars([(i,j) for i in M for j in N0], vtype=GRB.BINARY, name='y')
x = model.addVars([(i,j,k) for i in M for j in N0 for k in N0], vtype=GRB.BINARY, name='x')
C = model.addVars([i for i in N0], vtype=GRB.CONTINUOUS, name='C')
C_max = model.addVar(vtype=GRB.CONTINUOUS, name='C_max')

# Objective function
model.setObjective(C_max, GRB.MINIMIZE)

# Constraints
# 1. Each job is assigned to exactly one machine
for k in N:
    model.addConstr(gp.quicksum(y[i, k] for i in M) == 1, name=f'job_assignment_{k}')

# 2. Linking constraints for x and y
for k in N:
    for i in M:
        model.addConstr(y[i, k] == gp.quicksum(x[i, j, k] for j in N0 if j != k), name=f'linking1_{i}_{k}')
        model.addConstr(y[i, k] == gp.quicksum(x[i, k, j] for j in N0 if j != k), name=f'linking2_{i}_{k}')

# 3. Each machine can start at most one job from the dummy job
for i in M:
    model.addConstr(gp.quicksum(x[i, 0, k] for k in N) <= 1, name=f'dummy_start_{i}')

# 4. Precedence constraints
for i in M:
    for j in N0:
        for k in N:
            if j != k:
                model.addConstr(C[k] - C[j] + U * (1 - x[i, j, k]) >= setup_times[i][j][k] + processing_times[i][k], name=f'precedence_{i}_{j}_{k}')

# 5. C0 is zero
model.addConstr(C[0] == 0, name='C0_zero')

# 6. Completion times should be less than C_max
for j in N:
    model.addConstr(C[j] <= C_max, name=f'Cmax_bound_{j}')

# 7. Machine completion times constraints
for i in M:
    model.addConstr(
        gp.quicksum(setup_times[i][j][k] * x[i, j, k] for j in N0 for k in N if j != k) + 
        gp.quicksum(processing_times[i][k] * y[i, k] for k in N) <= C_max, 
        name=f'machine_completion_{i}'
    )

# Optimize the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print(f'Optimal C_max: {C_max.X}')
    for i in M:
        for k in N:
            if y[i, k].X > 0.5:
                print(f'Job {k} is assigned to machine {i}')
                for j in N0:
                    if x[i, j, k].X > 0.5:
                        print(f'    and follows job {j}')
else:
    print('No optimal solution found.')


Restricted license - for non-production use only - expires 2025-11-24


KeyError: 'Duplicate keys in Model.addVars()'